In [ ]:
import cv2
import mediapipe as mp
import math

def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def process_image(image_path, reference_height_cm=None):
    """
    image_path: путь к изображению с человеком в полный рост.
    reference_height_cm: известный рост человека (в см) для калибровки, если имеется.
    """
    # Загрузка изображения
    image = cv2.imread(image_path)
    if image is None:
        print("Не удалось загрузить изображение.")
        return
    
    image_height, image_width, _ = image.shape
    
    # Инициализация решения для обнаружения позы
    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        # Преобразование изображения в формат RGB
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.pose_landmarks:
            print("Не удалось обнаружить тело на изображении.")
            return
        
        landmarks = results.pose_landmarks.landmark
        
        # Расчёт "роста" в пикселях: разница между самой верхней и самой нижней точками
        y_coords = [landmark.y for landmark in landmarks]
        min_y = min(y_coords) * image_height
        max_y = max(y_coords) * image_height
        height_pixels = max_y - min_y
        
        # Примерный расчёт обхвата бёдер: расстояние между левым и правым тазовыми точками (landmarks 23 и 24)
        left_hip = (landmarks[23].x * image_width, landmarks[23].y * image_height)
        right_hip = (landmarks[24].x * image_width, landmarks[24].y * image_height)
        hips_width_pixels = calculate_distance(left_hip, right_hip)
        
        # Примерный расчёт ширины плеч: расстояние между точками левого и правого плеч (landmarks 11 и 12)
        left_shoulder = (landmarks[11].x * image_width, landmarks[11].y * image_height)
        right_shoulder = (landmarks[12].x * image_width, landmarks[12].y * image_height)
        shoulder_width_pixels = calculate_distance(left_shoulder, right_shoulder)
        
        # Если известен реальный рост (калибровка), вычисляем коэффициент преобразования (см/пиксель)
        if reference_height_cm:
            scale = reference_height_cm / height_pixels
            height_cm = height_pixels * scale
            hips_cm = hips_width_pixels * scale
            shoulder_cm = shoulder_width_pixels * scale
        else:
            scale = None
            height_cm = hips_cm = shoulder_cm = None
        
        # Оценка веса: здесь используется очень упрощённая модель (при допустимом BMI=22)
        if reference_height_cm:
            height_m = height_cm / 100
            estimated_weight_kg = 22 * (height_m ** 2)
        else:
            estimated_weight_kg = None
        
        # Вывод результатов
        print("Результаты измерений:")
        print(f"Рост в пикселях: {height_pixels:.2f}")
        if height_cm:
            print(f"Оценка роста: {height_cm:.2f} см")
        print(f"Ширина бёдер (расстояние между тазовыми точками) в пикселях: {hips_width_pixels:.2f}")
        if hips_cm:
            print(f"Оценка ширины бёдер: {hips_cm:.2f} см")
        print(f"Ширина плеч в пикселях: {shoulder_width_pixels:.2f}")
        if shoulder_cm:
            print(f"Оценка ширины плеч: {shoulder_cm:.2f} см")
        if estimated_weight_kg:
            print(f"Примерная оценка веса (при BMI=22): {estimated_weight_kg:.2f} кг")
        
        # Отрисовка найденных позных точек для визуализации
        annotated_image = image.copy()
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('Annotated Image', annotated_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Пример использования:
# Перед запуском убедитесь, что установлены библиотеки opencv-python и mediapipe:
# pip install opencv-python mediapipe

# Если известен рост человека (например, 175 см), можно передать его для калибровки:
process_image("/home/djden/Science/KAN/image_estimator/person.png", reference_height_cm=175)


I0000 00:00:1743332093.067423    8745 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 4
I0000 00:00:1743332093.102359    8901 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: SVGA3D; build: RELEASE;  LLVM;
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743332095.438967    8897 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743332096.439363    8899 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743332096.638494    8898 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Результаты измерений:
Рост в пикселях: 3383.89
Оценка роста: 175.00 см
Ширина бёдер (расстояние между тазовыми точками) в пикселях: 436.25
Оценка ширины бёдер: 22.56 см
Ширина плеч в пикселях: 699.86
Оценка ширины плеч: 36.19 см
Примерная оценка веса (при BMI=22): 67.38 кг


Only C and default locale supported with the posix collation implementation
Only C and default locale supported with the posix collation implementation
Case insensitive sorting unsupported in the posix collation implementation
Numeric mode unsupported in the posix collation implementation
